In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data 
data = pd.read_csv('merged_difference_cleaned_with_traits_and_values_cleaned_final_cleaned.csv')  

# 1. Display basic info and statistics
print("Dataset Information:")
data.info()

print("\nSummary Statistics:")
print(data.describe())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ch22o162               995 non-null    object 
 1   nomem_encr             995 non-null    int64  
 2   uy22a016               995 non-null    object 
 3   uy22a015               995 non-null    object 
 4   cr22o143               995 non-null    float64
 5   cr22o144               995 non-null    object 
 6   cs22o439               995 non-null    object 
 7   cs22o487               995 non-null    object 
 8   cs22o280               995 non-null    object 
 9   cs22o436               995 non-null    object 
 10  cs22o472               995 non-null    object 
 11  cs22o473               995 non-null    object 
 12  cs22o474               995 non-null    object 
 13  cs22o577               995 non-null    object 
 14  cs22o578               995 non-null  